
# Generate PC-Build Recommendation using ChatGPT4o

In [60]:
import json

# 更新 component_map，將 Fan 對應為 Case Fan
component_map = {
    "Mother board": "Motherboard",
    "Case": "Case",
    "CPU": "CPU",
    "GPU": "GPU",
    "Memory": "Memory",
    "Device": "SSD",
    "Power": "Power Supply",
    "Fan": "Case Fan"  # 將 Fan 視為機殼風扇
}

def format_recommendation(recommendation_dict):
    """
    將建議的字典資料轉換成字串輸出，格式如：
    CPU: <name> | price: <price>
    ...
    總價: <sum>
    """
    lines = []
    total_price = 0
    # 新增 "Case Fan" 到輸出順序
    order = ["CPU", "Motherboard", "Memory", "GPU", "SSD", "Power Supply", "Case", "Case Fan"]
    for comp in order:
        if comp in recommendation_dict:
            comp_info = recommendation_dict[comp]
            name = comp_info.get("name", "N/A")
            price = int(comp_info.get("price", "0"))
            total_price += price
            lines.append(f"{comp}: {name} | price: {price}")
    lines.append(f"總價: {total_price}")
    return "\n".join(lines)


def generate_gpt4o_recommendation(query, llm: ChatOpenAI):
    """
    呼叫 GPT-4 模型根據 query 產生推薦清單，
    並使用 one-shot 提示要求特定的 JSON 格式輸出。
    """

    # 一次範例 One-shot 提示 (示意，可以調整成更符合您需求的範例)
    # 注意內容：Fan 是 "Case Fan"
    example_output = {
        "CPU": {
            "reason": "適合該需求場景的高效處理器，提供足夠運算效能。",
            "name": "Intel Core i5-12400",
            "price": "5790"
        },
        "Motherboard": {
            "reason": "穩定耐用，支援所選CPU及其他組件。",
            "name": "ASUS PRIME B660M-A",
            "price": "3990"
        },
        "Memory": {
            "reason": "足夠容量與適當頻率，支持多工與目標應用。",
            "name": "G.SKILL Ripjaws V 16GB (2x8GB) DDR4-3200",
            "price": "1800"
        },
        "GPU": {
            "reason": "滿足遊戲或加速需求，提供流暢影像與運算。",
            "name": "NVIDIA GeForce RTX 3060",
            "price": "8500"
        },
        "SSD": {
            "reason": "快速存取資料，提高系統整體響應速度。",
            "name": "Samsung 970 EVO Plus 1TB",
            "price": "2200"
        },
        "Power Supply": {
            "reason": "穩定供電，確保系統長期穩定運行。",
            "name": "Corsair RM650x 80+ Gold",
            "price": "3200"
        },
        "Case": {
            "reason": "良好結構與散熱空間，易於安裝和維護。",
            "name": "Fractal Design Meshify C",
            "price": "2500"
        },
        "CPU Cooler": {
            "reason": "足夠散熱能力，確保CPU在高負載下穩定。",
            "name": "Cooler Master Hyper 212",
            "price": "1000"
        },
        "Case Fan": {
            "reason": "提供額外氣流，提高機殼內部散熱效率。",
            "name": "Cooler Master SickleFlow 120",
            "price": "200"
        }
    }

    system_prompt = f"""你是一位組裝電腦的專家，請根據以下使用者需求提供一份電腦組裝清單。
    請用 JSON 格式輸出，並包含以下元件的資訊：
    CPU, Motherboard, Memory, GPU, SSD, Power Supply, Case, CPU Cooler, Case Fan

    每個元件需含有 `name`, `price`, `reason` 三個欄位（reason為簡短原因說明）。
    Fan已對應為 Case Fan。

    使用者需求：
    {query}

    以下為範例輸出格式（One-shot），請仿照此結構但內容請重新推薦適合的產品（根據上述query）：
    ```json
    {json.dumps(example_output, ensure_ascii=False, indent=2)}
    請直接輸出JSON格式結果，勿包含多餘說明文字。 """
    messages = [
    SystemMessage(content=system_prompt),
    ]
    print (SystemMessage(content=system_prompt))
    
    
    response = llm.invoke(messages)

    print (response)

    pattern = r'```json\s*(\{[\s\S]*?\})\s*```'

    match = re.search(pattern, response.content)
    if match:
        json_str = match.group(1)  # 取得 JSON 區塊
        try:
            result = json.loads(json_str)
            print(result)
        except json.JSONDecodeError as e:
            print("JSON解析失敗:", e)
            result = {}

    else:
        print("未找到 JSON 區塊")
        result = {}
    return result
    
    


def main():
    buildmate_file = "./results.json"
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

    with open(buildmate_file, "r", encoding="utf-8") as f:
        buildmate_data = json.load(f)
    print (buildmate_data)
    final_results = []
    
    question_id = 1
    
    for query, components in buildmate_data.items():
        # 將 BuildMate 推薦清單轉成標準格式
        buildmate_recom = {}
        for original_comp, comp_info in components.items():
            mapped_comp = component_map.get(original_comp, original_comp)
            buildmate_recom[mapped_comp] = {
                "name": comp_info.get("name", "N/A"),
                "price": comp_info.get("price", "0"),
                "reason": comp_info.get("reason", "")
            }

        # 產生 gpt4o 推薦清單
        gpt4o_recom = generate_gpt4o_recommendation(query, llm)

        # 將兩者的清單格式化為字串
        gpt4o_str = format_recommendation(gpt4o_recom)
        buildmate_str = format_recommendation(buildmate_recom)

        # 整理成最終 output 格式
        final_results.append({
            "question_id": question_id,
            "question": query,
            "gpt4o_recommendation": gpt4o_str,
            "buildmate_recommendation": buildmate_str
        })
        if question_id == 10:
            break
        question_id += 1

        

    # 輸出 final results
    with open("final_output.json", "w", encoding="utf-8") as out_f:
        json.dump(final_results, out_f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    main()


{'請問30000元內能組一台什麼樣的主機，主要是玩LOL和看Netflix用？': {'Mother board': {'reason': '這款主機板支援AMD處理器，穩定性高，適合遊戲及日常使用。', 'name': 'MSI B450M PRO-VDH MAX', 'price': '3000'}, 'Case': {'reason': '這個配置能夠流暢運行LOL和Netflix，並具備升級潛力。', 'name': 'Cooler Master MasterBox Q300L', 'price': '1200'}, 'CPU': {'reason': '性能優越，適合遊戲及多媒體使用，具備集成顯示卡可省去獨立顯卡的需求。', 'name': 'AMD Ryzen 5 5600G', 'price': '7500'}, 'GPU': {'reason': '這款顯示卡性能足以應對LOL，且適合多媒體播放。', 'name': 'NVIDIA GeForce GTX 1650', 'price': '8000'}, 'Memory': {'reason': '這款記憶體提供穩定的性能，適合遊戲和串流觀看，性價比高。', 'name': 'Corsair Vengeance LPX 16GB (2 x 8GB) DDR4 3200MHz', 'price': '2000'}, 'Device': {'reason': '性能穩定，適合LOL及影片串流，性價比高。', 'name': 'MSI B450M PRO-VDH MAX', 'price': '3000'}, 'Power': {'reason': '穩定供電，保證系統性能，並具備良好的散熱效果。', 'name': 'Corsair CV450 450W 80 PLUS Bronze', 'price': '1500'}, 'Fan': {'reason': '提供良好的散熱性能，適合長時間運行遊戲和影音串流。', 'name': 'Cooler Master Hyper 212 LED', 'price': '1200'}}, '有推薦的組合嗎？預算大概在50000元，要能處理攝影後製和輕遊戲，如：軒轅劍7。': {'Mother board': {'reason': '適合攝影後製及輕遊戲，穩定性

# CouncilEval
- A collaborative evaluation framework where multiple expert agents convene to assess, compare, and refine recommendations. By simulating a panel of specialists—each focusing on different performance criteria—CouncilEval ensures well-rounded, consensus-driven decisions on system recommendations, ultimately delivering more balanced, reliable, and user-focused outcomes.

In [2]:
from typing import Dict, List, TypedDict
from langgraph.graph import StateGraph, END, START
from typing import Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
import json
from langchain_openai import ChatOpenAI
from typing import Optional
import re
import rich
def convert_messages_to_serializable(obj):
    if isinstance(obj, dict):
        # 遞迴處理字典
        return {k: convert_messages_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        # 遞迴處理列表
        return [convert_messages_to_serializable(v) for v in obj]
    elif isinstance(obj, HumanMessage) or isinstance(obj, SystemMessage):
        # 將 Message 物件轉成字典格式
        # 假設 content 前面有 "Speaker: 內容" 格式，可以分離
        parts = obj.content.split(":", 1)
        if len(parts) == 2:
            speaker = parts[0].strip()
            message = parts[1].strip()
        else:
            speaker = "unknown"
            message = obj.content
        
        return {
            "speaker": speaker,
            "message": message
        }
    else:
        # 對於非字典/列表/Message物件，原樣返回
        return obj

class EvaluationState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], "聊天歷史"]
    next_agent: str
    # 基本資料
    question_id: int
    question: str
    gpt4o_recommendation: str
    buildmate_recommendation: str
    agent_scores: Dict[str, Dict[str, int]]
    round_count: int     # 用於紀錄第幾輪
    max_rounds: int       # 最大回合數
    consensus_reached: bool  # 是否已達成共識

def create_agent(name: str, llm):
    # new 
    system_prompts = {
    "Agent1": """你是電腦硬體顧問，專精於「效能」與「系統穩定」層面。
請從處理器、顯示卡、記憶體、儲存裝置的整體效能，以及散熱方案、電源品質、系統組裝平衡度，來比較兩個推薦清單的效能與穩定度。

請提供：
[對gpt4o評價]: (0~10)
[對BuildMate評價]: (0~10)
[簡短原因]: (以專業觀點精簡描述)

注意事項：
1. 原因的說明請限制在50個字以內。
""",

    "Agent2": """你是電腦硬體顧問，專精於「價格/效能比」(CP值)。
請根據兩個推薦清單中各零件的價位與提供效能是否相稱、預算是否用在刀口上，評估CP值高低。

請提供：
[對gpt4o評價]: (0~10)
[對BuildMate評價]: (0~10)
[簡短原因]: (以專業觀點精簡描述)

注意事項：
1. 原因的說明請限制在50個字以內。
""",

    "Agent3": """你是電腦硬體顧問，專精於「需求符合度」。
1.請根據使用者的需求場景（例如遊戲、剪輯、多工、深度學習、家庭多媒體等），判斷各推薦清單是否選用合適的CPU/GPU、記憶體容量、儲存空間及其他週邊，並評估清單是否過度或不足。
2.請評估使用者的總預算與清單總價是否接近，在使用者預算合理下，應該要充分運用預算。
3.請評估清單是否與使用者的需求相符，例如零組件外觀，主機大小，保固等各式需求。
4.請評估使用者的特殊考量事項是否被滿足，例如：省電。

請提供：
[對gpt4o評價]: (0~10)
[對BuildMate評價]: (0~10)
[簡短原因]: (以專業觀點精簡描述)

注意事項：
1. 原因的說明請限制在50個字以內。
""",

    "Coordinator": """你是最終協調者，已獲得3位專家針對效能與穩定、CP值、需求符合度的評分。
如果尚未達成共識，請只簡短回應「未達共識」。
如果已達共識，請回應「已達成共識」並指出哪一方較佳(不超過50字)。"""
}


    def agent_function(state: EvaluationState):
        messages = list(state["messages"])
        question = state["question"]
        gpt4o_recom = state["gpt4o_recommendation"]
        buildmate_recom = state["buildmate_recommendation"]

        system_msg = SystemMessage(content=system_prompts[name])
        user_prompt = f"""(第{state['round_count']}輪)
問題: {question}

--- gpt4o的組裝清單 ---
{gpt4o_recom}

--- BuildMate的組裝清單 ---
{buildmate_recom}
"""

        conversation = [system_msg]
        conversation.extend(messages)
        conversation.append(HumanMessage(content=user_prompt))

        llm_response = llm.invoke(conversation)
        content_with_speaker = f"{name}: {llm_response.content}"
        rich.print( content_with_speaker)
        if name in ("Agent1","Agent2","Agent3"):
            gpt4o_match = re.search(r"\[對gpt4o評價\]:\s*(\d+)", llm_response.content)
            bm_match = re.search(r"\[對BuildMate評價\]:\s*(\d+)", llm_response.content)
            pattern = r"\[簡短原因\]:\s*(.+)"
            reason_match = re.search(pattern, llm_response.content)
            reason = reason_match.group(1)

            if gpt4o_match and bm_match:
                gpt4o_score = int(gpt4o_match.group(1))
                bm_score = int(bm_match.group(1))
                # 將分數存入
                state["agent_scores"][f"{name}_round{state['round_count']}"] = {
                    "gpt4o_score": gpt4o_score,
                    "buildmate_score": bm_score,
                    "reason": reason
                }
                #print (f'gpt score: {gpt4o_score}, buildmate score: {bm_score}')

        if name == "Coordinator":
            # 假設協調者若尚未達成共識則回應「未達共識」四字，如果達成共識就「已達成共識」
            if "已達成共識" in llm_response.content:
                state["consensus_reached"] = True
            else:
                state["consensus_reached"] = False

        new_message = HumanMessage(content=content_with_speaker)

        return {
            "messages": [*messages, new_message],
            "next_agent": decide_next_speaker(name, state),
            "question_id": state["question_id"],
            "question": state["question"],
            "gpt4o_recommendation": state["gpt4o_recommendation"],
            "buildmate_recommendation": state["buildmate_recommendation"],
            "agent_scores": state["agent_scores"],
            "round_count": state["round_count"],
            "max_rounds": state["max_rounds"],
            "consensus_reached": state["consensus_reached"]
        }
    return agent_function


def decide_next_speaker(current_speaker: str, state: EvaluationState) -> str:
    if current_speaker == "Agent1":
        return "Agent2"
    elif current_speaker == "Agent2":
        return "Agent3"
    elif current_speaker == "Agent3":
        return "Coordinator"
    elif current_speaker == "Coordinator":
        # 即使已達共識，但如果 round_count < 2，仍然強制進入下一輪
        if state["round_count"] < 2:
            # 即使協調者表示「已達成共識」，也不要結束
            state["round_count"] += 1
            state["consensus_reached"] = False  # 忽略本輪的共識結果，強制再跑一輪
            return "Agent1"
        else:
            # 現在已經執行到第二輪或更多
            # 這裡才根據共識及最大回合數決定是否結束
            if state["consensus_reached"] or state["round_count"] >= state["max_rounds"]:
                return END
            else:
                state["round_count"] += 1
                return "Agent1"
    else:
        return END

def create_workflow(llm):
    workflow = StateGraph(EvaluationState)
    workflow.add_node("Agent1", create_agent("Agent1", llm))
    workflow.add_node("Agent2", create_agent("Agent2", llm))
    workflow.add_node("Agent3", create_agent("Agent3", llm))
    workflow.add_node("Coordinator", create_agent("Coordinator", llm))

    def router(state: EvaluationState) -> str:
        return state["next_agent"]

    workflow.add_conditional_edges("Agent1", router, {"Agent2": "Agent2", END: END})
    workflow.add_conditional_edges("Agent2", router, {"Agent3": "Agent3", END: END})
    workflow.add_conditional_edges("Agent3", router, {"Coordinator": "Coordinator", END: END})
    workflow.add_conditional_edges("Coordinator", router, {"Agent1": "Agent1", END: END})

    workflow.add_edge(START, "Agent1")

    return workflow.compile()



recommendation_file = './final_output.json'
with open(recommendation_file, "r", encoding="utf-8") as f:
        recommendation_data  = json.load(f)
        final_report = []  # 用來儲存所有 question 的評估結果

        for sample_data in recommendation_data:

            rich.print(sample_data)
            llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0.3)
            app = create_workflow(llm)

            initial_state = {
                "messages": [],
                "next_agent": "Agent1",
                "question_id": sample_data["question_id"],
                "question": sample_data["question"],
                "gpt4o_recommendation": sample_data["gpt4o_recommendation"],
                "buildmate_recommendation": sample_data["buildmate_recommendation"],
                "agent_scores": {},
                "round_count": 1,
                "max_rounds": 3,  #最多3回合
                "consensus_reached": False
            }

            final_state = None
            i = 0 
            for output in app.stream(initial_state):
                print (f"======")
                final_state = output
                i = i + 1
            print (final_state)
            serializable_state = dict(final_state)
            state = final_state["Coordinator"]
            question_id = state["question_id"]
            question = state["question"]
            gpt4o_recom = state["gpt4o_recommendation"]
            buildmate_recom = state["buildmate_recommendation"]
            agent_scores = state["agent_scores"]


            gpt4o_scores_all = []
            buildmate_scores_all = []
            rounds_data = {}

            for key, scores in agent_scores.items():
                m = re.match(r"(Agent\d+)_round(\d+)", key)
                if m:
                    agent_name = m.group(1)
                    round_num = int(m.group(2))
                    if round_num not in rounds_data:
                        rounds_data[round_num] = {}
                    rounds_data[round_num][agent_name] = scores
                    gpt4o_scores_all.append(scores["gpt4o_score"])
                    buildmate_scores_all.append(scores["buildmate_score"])

            if gpt4o_scores_all:
                gpt4o_avg = sum(gpt4o_scores_all) / len(gpt4o_scores_all)
            else:
                gpt4o_avg = 0

            if buildmate_scores_all:
                buildmate_avg = sum(buildmate_scores_all) / len(buildmate_scores_all)
            else:
                buildmate_avg = 0

            # 判斷誰贏 (1: BuildMate 贏; 0: BuildMate 輸)
            winner = 1 if buildmate_avg > gpt4o_avg else 0

            # 整理 round 資訊
            rounds_list = []
            for r in sorted(rounds_data.keys()):
                round_info = {"round_number": r}
                for agent_name, sc in rounds_data[r].items():
                    round_info[agent_name] = {
                        "gpt4o_score": sc["gpt4o_score"],
                        "buildmate_score": sc["buildmate_score"],
                        "reason": sc["reason"]
                    }
                rounds_list.append(round_info)

            report_item = {
                "question_id": question_id,
                "question": question,
                "buildmate_recommendation":buildmate_recom,
                "gpt4o_recommendation": gpt4o_recom,
                "winner": winner,
                "buildmate_avg_score": buildmate_avg,
                "gpt4o_avg_score": gpt4o_avg,
                "rounds": rounds_list
            }

            print(json.dumps(report_item, ensure_ascii=False, indent=2))
            final_report.append(report_item)

            #serializable_result = convert_messages_to_serializable(final_state)
with open("evaluation_report_budget_reason.json", "w", encoding="utf-8") as out_f:
    json.dump(final_report, out_f, ensure_ascii=False, indent=2)





{
    'question_id': 1,
    'question': '請問30000元內能組一台什麼樣的主機，主要是玩LOL和看Netflix用，機殼要小且多彩？',
    'gpt4o_recommendation': 'CPU: AMD Ryzen 5 5600G | price: 5500\nMotherboard: ASUS TUF Gaming B550M-PLUS | price:
3500\nMemory: Corsair Vengeance LPX 16GB (2x8GB) DDR4-3200 | price: 1800\nGPU: AMD Radeon Graphics (integrated) | 
price: 0\nSSD: Crucial P3 500GB NVMe SSD | price: 1500\nPower Supply: Thermaltake Smart 500W 80+ | price: 
1500\nCase: Cooler Master MasterBox Q300L | price: 1200\nCase Fan: ARCTIC P12 PWM PST 120mm | price: 300\n總價: 
15300',
    'buildmate_recommendation': 'CPU: AMD Ryzen 5 5600G | price: 7500\nMotherboard: MSI B450M PRO-VDH MAX | price: 
3000\nMemory: Corsair Vengeance LPX 16GB (2 x 8GB) DDR4 3200MHz | price: 2000\nGPU: NVIDIA GeForce GTX 1650 | 
price: 8000\nSSD: MSI B450M PRO-VDH MAX | price: 3000\nPower Supply: Corsair CV450 450W 80 PLUS Bronze | price: 
1500\nCase: Cooler Master MasterBox Q300L | price: 1200\nCase Fan: Cooler Master Hyper 212 LED | price: 1200\n總價:
27400'
}

Agent1: [對gpt4o評價]: 7  
[對BuildMate評價]: 8  

[簡短原因]:  
gpt4o方案更省預算，適合LOL和Netflix，但效能受限於整合顯示卡；BuildMate方案有GTX 
1650，效能更高，但價格接近預算上限，散熱和電源稍弱於理想配置。

Agent2: [對gpt4o評價]: 8  
[對BuildMate評價]: 7  

[簡短原因]:  
gpt4o方案以整合顯卡為主，效能足夠應付LOL和Netflix，價格低且預算分配合理；BuildMate方案有獨顯，效能更高但價格偏高，
且CPU和GPU搭配性價比不佳。

Agent3: [對gpt4o評價]: 8  
[對BuildMate評價]: 7  

[簡短原因]:  
gpt4o方案效能足夠應付LOL和Netflix，價格低且預算分配合理，但整合顯卡效能有限；BuildMate方案有獨顯，效能更高，但價格
接近預算上限，且CPU與GPU搭配性價比不佳。

Coordinator: 已達成共識，gpt4o較佳：效能足夠應付LOL和Netflix，價格低且預算分配合理，適合需求。

...

Agent1: [對gpt4o評價]: 9  
[對BuildMate評價]: 7  

[簡短原因]:  
gpt4o配置平衡，RX 6700 
XT效能更適合VR遊戲，且價格低於預算，散熱與電源穩定；BuildMate過度投資於CPU與主機板，GPU效能不足，整體效能不均衡。

Agent2: [對gpt4o評價]: 9  
[對BuildMate評價]: 6  

[簡短原因]:  
gpt4o配置平衡，RX 6700 XT效能強於RTX 
3060，適合VR遊戲，且價格遠低於預算；BuildMate過度投資於CPU與主機板，GPU效能不足，預算分配不合理。

Agent3: [對gpt4o評價]: 9  
[對BuildMate評價]: 6  

[簡短原因]:  
gpt4o配置平衡，RX 6700 
XT效能適合VR遊戲，價格僅用了一半預算，具升級空間；BuildMate過度投資於CPU與主機板，GPU效能不足，對VR遊戲不理想，預算
分配不均。

Coordinator: 已達成共識，gpt4o較佳：效能與穩定性平衡，GPU適合VR遊戲，價格僅用一半預算，具升級空間，CP值更高。

{'Coordinator': {'messages': [HumanMessage(content='Agent1: [對gpt4o評價]: 8  \n[對BuildMate評價]: 6  \n\n[簡短原因]:  \ngpt4o在效能與穩定性上更平衡，RX 6700 XT效能優於RTX 3060，且電源與散熱方案穩定；BuildMate的配置過於偏重CPU，GPU效能不足，且價格失衡。', additional_kwargs={}, response_metadata={}), HumanMessage(content='Agent2: [對gpt4o評價]: 9  \n[對BuildMate評價]: 6  \n\n[簡短原因]:  \ngpt4o配置平衡，RX 6700 XT效能適合VR遊戲，且價格合理；BuildMate過度投資於CPU與主機板，GPU效能不足，對VR遊戲不夠理想，預算分配不均。', additional_kwargs={}, response_metadata={}), HumanMessage(content='Agent3: [對gpt4o評價]: 9  \n[對BuildMate評價]: 6  \n\n[簡短原因]:  \ngpt4o配置平衡，RX 6700 XT效能適合VR遊戲，且價格遠低於預算，具升級空間；BuildMate過度投資於CPU與主機板，GPU效能不足，對VR遊戲不夠理想，預算分配不均。', additional_kwargs={}, response_metadata={}), HumanMessage(content='Coordinator: 已達成共識，gpt4o較佳：效能與穩定性平衡，GPU適合VR遊戲，價格更具CP值，且符合需求。', additional_kwargs={}, response_metadata={}), HumanMessage(content='Agent1: [對gpt4o評價]: 9  \n[對BuildMate評價]: 7  \n\n[簡短原因]:  \ngpt4o配置平衡，RX 6700 XT效能更適合VR遊戲，且價格低於預算，散熱與電源穩定；BuildMate過度投資於CPU與主機板，GPU效能不足，整體效能不均衡。', additional_kwargs={}, respo

In [ ]:
sample_data = {
"question_id": 1,
"question": "我有30K預算組遊戲機",
"gpt4o_recommendation": """CPU: Intel Core i5-12400 | price: 5,790
Motherboard: 華擎 B660M PG Riptide | price: 3,990
Memory: 十銨 T-Force Vulcan Z DDR4-3200 32GB (16GB*2) | price: 1,899
GPU: 微星 RTX 3060 VENTUS 2X 12G OC | price: 8,490
SSD: 金士頓 KC3000 1TB M.2 PCIe 4.0 | price: 1,950
Power Supply: NZXT C650 80+金牌 全模組 | price: 2,990
Case: Antec P20CE | price: 2,390
CPU Cooler: 利民 Assassin Spirit 120 Plus V2 | price: 790
總價: 28,299""",
        "buildmate_recommendation": """CPU: AMD Ryzen 5 5600 | price: 5,200
Motherboard: 微星 B550M PRO-VDH | price: 3,400
Memory: 金士頓 FURY DDR4-3200 16GB*2 | price: 1,700
GPU: 技嘉 GeForce RTX 3060 EAGLE 12G | price: 8,300
SSD: 鎂光 P5 Plus 1TB PCIe 4.0 | price: 2,000
Power Supply: 恆能源 650W 80+金牌 | price: 2,700
Case: Cooler Master MasterBox MB400L | price: 1,800
CPU Cooler: 追風者 PH-TC12LS | price: 700
總價: 25,800"""
}